In [2]:
import os
from typing import Literal
from openai import OpenAI, AzureOpenAI
import requests
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
# Validate required environment variables
required_vars = ["AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_VERSION"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    raise EnvironmentError(f"Missing required environment variables: {', '.join(missing_vars)}")

In [6]:


try:
    azure_client = AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_version=os.getenv("AZURE_OPENAI_API_VERSION")
    )
    # Make a completions call
    response = azure_client.chat.completions.create(
        model="gpt-4o",  
        messages=[{"role": "user", "content": "hi"}]
    )
    print("Successfully connected ")
    print("Response:", response.choices[0].message.content)
except Exception as e:
    print(f"Failed connecting: {str(e)}")

Successfully connected 
Response: Hello! How can I assist you today? 😊


In [13]:
def check_service_status(service: Literal["openai", "azure"] = "openai") -> bool:
    """
    Check if OpenAI or Azure OpenAI API service is operational.
    
    Args:
        service (Literal["openai", "azure"]): The service to check status for.
    
    Returns:
        bool: True if API is operational (no outages), False otherwise
    """
    if service == "openai":
        url = "https://status.openai.com/api/v2/summary.json"
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            # Find the API component
            for component in data["components"]:
                if component["name"].startswith("API"):
                    # operational means everything is working fine
                    return component["status"] == "operational"        
            # If API component not found, assume there's an issue
            return False
            
        except requests.exceptions.RequestException as e:
            print(f"Error checking OpenAI status: {e}")
            return False
            
    elif service == "azure":
        # Azure OpenAI status is part of Azure's overall status
        # We can check the Azure status page for the specific region
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        if not azure_endpoint:
            print("Azure OpenAI endpoint not configured")
            return False
            
        # Extract region from endpoint (e.g., https://<resource>.openai.azure.com/)
        try:
            region = azure_endpoint.split('.')[1]  # Gets 'openai' from the endpoint
            url = f"https://status.azure.com/en-us/status"
            try:
                response = requests.get(url)
                response.raise_for_status()
                return "Azure OpenAI" not in response.text.lower()
            except requests.exceptions.RequestException as e:
                print(f"Error checking Azure status: {e}")
                return False
        except IndexError:
            print("Invalid Azure OpenAI endpoint format")
            return False
            
    else:
        raise NotImplementedError(f"Status check for {service} not implemented")


In [14]:
# Check OpenAI status
openai_status = check_service_status("openai")

In [15]:
openai_status

True

In [16]:
# Check Azure OpenAI status
azure_status = check_service_status("azure")

In [17]:
azure_status

True